In [11]:
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb#scrollTo=3SDhbo8lOBQv

import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

import tempfile
model_dir = tempfile.mkdtemp()

In [12]:
data_dir = pathlib.Path("./data/dogs-vs-cats/")

In [13]:
image_count = len(list(data_dir.glob('*/*.jpg')))

In [14]:
SEED = 0
EPOCH_SIZE = 5
BATCH_SIZE = 32
BUFFER_SIZE=200
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = image_count//BATCH_SIZE
CLASS_NAMES = np.array(["cat", "dog"])

tf.random.set_seed(SEED)

In [15]:
def get_label(path):
    s_path = tf.strings.split(path, os.path.sep)[-1]
    label = tf.strings.split(s_path, ".")[0]
    if label == "cat":
        return 0
    else:
        return 1

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(path):
    label = get_label(path)
    img = tf.io.read_file(path)
    img = decode_img(img)
    return img, label

def scale_image(image, label):
    image = tf.image.per_image_standardization(image)
    return image, label

In [16]:
def input_fn(path):
    list_ds = tf.data.Dataset.list_files(str(path))
    
    return list_ds \
        .map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
        .map(scale_image) \
        .shuffle(BUFFER_SIZE) \
        .batch(BATCH_SIZE) \
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE) \
        .repeat(EPOCH_SIZE)

for image, label in input_fn(data_dir/'*/*').take(5):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.shape)

Image shape:  (32, 224, 224, 3)
Label:  (32,)
Image shape:  (32, 224, 224, 3)
Label:  (32,)
Image shape:  (32, 224, 224, 3)
Label:  (32,)
Image shape:  (32, 224, 224, 3)
Label:  (32,)
Image shape:  (32, 224, 224, 3)
Label:  (32,)


In [17]:
train_ds = input_fn(data_dir/'*/*')

In [18]:
def model_fn():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, kernel_size=(5, 5), input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(2), input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation='relu'),
        tf.keras.layers.Dense(84, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.RMSprop(),
                  loss=tf.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=["accuracy"])
    
    return model

In [19]:
model = model_fn()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 16)        2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               1505400   
_________________________________________________________________
dense_3 (Dense)              (None, 84)               

In [ ]:
history = model.fit(train_ds, epochs = EPOCH_SIZE, steps_per_epoch = STEPS_PER_EPOCH, use_multiprocessing=True, workers=7)

Train for 781 steps
Epoch 1/5
781/781 [==============================] - 310s 397ms/step - loss: 0.5704 - accuracy: 0.6937
Epoch 2/5
781/781 [==============================] - 312s 399ms/step - loss: 0.4401 - accuracy: 0.7897
Epoch 3/5
781/781 [==============================] - 316s 405ms/step - loss: 0.3420 - accuracy: 0.8462
Epoch 4/5
132/781 [====>.........................] - ETA: 4:20 - loss: 0.2178 - accuracy: 0.9074

In [ ]:
acc = history.history['accuracy']
loss=history.history['loss']

In [ ]:
epochs_range = range(EPOCH_SIZE)
# plt.subplot(1, 2, 2)
plt.plot(epochs_range, acc, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.evaluate(train_ds, steps=100)